In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.linear_model import Ridge

In [2]:
#!pip install fsspec
#!pip install gcsfs

# Opening files

In [3]:
### data not to be used
# df_transactions = pd.read_csv(path+'transactions.csv')
# df_oil = pd.read_csv(path+'oil.csv')
# df_holidays = pd.read_csv(path+'holidays_events.csv')

In [4]:
path = 'gs://business-case/'

In [5]:
df_train = pd.read_csv(path+'train.csv')
df_train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [6]:
df_test = pd.read_csv(path+'test.csv')
df_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [7]:
df_stores = pd.read_csv(path+"stores.csv")
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


# Pipeline Workflow

## Data Merging

In [8]:
### merging keys if upgrade is needed
# transactions >>> on=['date','store_nbr'])
# oil >>> on='date'

In [9]:
def DataMerging(csv_left, csv_stores):
    
    csv_stores = csv_stores[['store_nbr','city']]
    df = pd.merge(csv_left, csv_stores, how='left', on='store_nbr')
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    #df['year'] = pd.to_datetime(df['date']).dt.year
    df = df.drop(columns=['date', 'id'])
    
    return df    

In [10]:
df_train_merged = DataMerging(df_train,df_stores)
df_train_merged

,store_nbr,family,sales,onpromotion,city,year
0,1,AUTOMOTIVE,0.000,0,Quito,2013
1,1,BABY CARE,0.000,0,Quito,2013
2,1,BEAUTY,0.000,0,Quito,2013
3,1,BEVERAGES,0.000,0,Quito,2013
4,1,BOOKS,0.000,0,Quito,2013
...,...,...,...,...,...,...
3000883,9,POULTRY,438.133,0,Quito,2017
3000884,9,PREPARED FOODS,154.553,1,Quito,2017
3000885,9,PRODUCE,2419.729,148,Quito,2017
3000886,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,2017


In [11]:
df_test_merged = DataMerging(df_test,df_stores)
df_test_merged

,store_nbr,family,onpromotion,city,year
0,1,AUTOMOTIVE,0,Quito,2017
1,1,BABY CARE,0,Quito,2017
2,1,BEAUTY,2,Quito,2017
3,1,BEVERAGES,20,Quito,2017
4,1,BOOKS,0,Quito,2017
...,...,...,...,...,...
28507,9,POULTRY,1,Quito,2017
28508,9,PREPARED FOODS,0,Quito,2017
28509,9,PRODUCE,1,Quito,2017
28510,9,SCHOOL AND OFFICE SUPPLIES,9,Quito,2017


In [12]:
# sns.set(style='white')
# sns.pairplot(df_train_merged, height=2)
# plt.show()

## Preprocessing

In [13]:
X_train = df_train_merged.drop(columns='sales')
y_train = df_train_merged['sales']

In [14]:
num_transformer = Pipeline([
    ('scaler', StandardScaler())
])

cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

preprocessor = ColumnTransformer([
        ('num_tr', num_transformer, ['onpromotion']),
        ('cat_tr', cat_transformer, ['store_nbr', 'family', 'city'])
    #],remainder='passthrough'
])

In [15]:
preprocessor

ColumnTransformer(transformers=[('num_tr',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['onpromotion']),
                                ('cat_tr',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['store_nbr', 'family', 'city'])])

In [16]:
pipe = make_pipeline(preprocessor, Ridge())

In [ ]:
# Train pipeline
pipe.fit(X_train, y_train)

In [ ]:
X_test = DataMerging(df_test,df_stores)
X_test

In [ ]:
### preprocessor outputs

# X_train_transformed = preprocessor.fit_transform(X_train)
# display(X_train.head(3))
# display(pd.DataFrame(X_train_transformed).head(3))

# SimpleImputer.get_feature_names_out = (lambda self, names=None: self.feature_names_in_)
# X_train_transformed = pd.DataFrame(X_train_transformed,
                                        #columns=preprocessor.get_feature_names_out()
                                    #)
        
# X_train_transformed.head()

In [ ]:
# Make predictions
y_pred = pipe.predict(X_test)

In [ ]:
# Score model
pipe.score(X_train,y_train)

In [ ]:
sns.histplot(y_pred)

In [ ]:
y_test - y_pred to print the residual